# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [2]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [3]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [4]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [5]:
from __future__ import division

f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [1]:
#   Базовая функция должна возвращать функцию, применение 
# которой к x0 вернет нужное значение.
#   Рассматривая частные случаи с пустым и одноэлементным 
# списками, можно прийти к таким решениям:

def foldl2(f, x0, lst):
    return foldr(lambda elem, res_func: lambda e: res_func(f(e, elem)), lambda e: e, lst)(x0)

In [2]:
def foldr2(f, x0, lst):
    return foldl(lambda res_func, elem: lambda e: res_func(f(elem, e)), lambda e: e, lst)(x0)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [1]:
import random

# таблица хеш значений для символов. 
# Для юникодных символов можно использовать "настоящую" хеш функцию.
CH_HASH = [random.randint(1, 2**63) for i in xrange(256)]

def check_inv(a, b):
    """
          При движении окна длины len(b) по строке а можно эффективно вычислять 
        сумму хешей символов попадающих в окно. 
          Если эта сумма не совпадает с вычисленной заранее аналогичной суммой для 
        строки b. То окно заведомо не содержит перестановку символов b. 
          Если суммы совпадают, то сортируем символы окна и сравниваем с заранее 
        отсортироваными символами строки b.
        
          При условии len(b) << len(a) и если коллизий мало, временем точной 
        проверки можно пренебреч.Тогда сложность алгоритма порядка len(a). 
        
          В худшем случве (хеши всех символов равны), будет выполнено len(a) сортировок 
        содержимого окна что даст сложность len(a) * len(b) * ln(len(b)).
        
          Эта задача очень похожа на задачу поиска подстроки, что позволяет придумывать 
        алгоритмы руковдствуясь похожими соображениями. 
          Предложеный алгоритм, похожий на алгоритм Рабина - Карпа для поиска 
        подстроки, представляется мне одним из самых простых в реализации.
    """
    if len(a) < len(b): return False
    
    # Необходимые константы.
    b_sum = sum(CH_HASH[ord(c)] for c in b)
    b_sort = "".join(sorted(b))
    
    # Проверка на то, что окно длины len(b) и смещением offset от 
    # начала строки a есть перестановка символов строки b.
    def check(offset, w_sum): 
        if w_sum != b_sum:
            return False
        return  "".join(sorted(a[offset:offset+len(b)])) == b_sort
        
    # Начальное положение окна.
    offset = 0
    w_sum = sum(CH_HASH[ord(c)] for c in a[:len(b)])
    
    # Окно в начальном положении не содержит перестановку.
    if check(offset, w_sum): 
        return True
        
    # out_ch - Крайне левый символ окна.
    # in_ch - Первый символ за правой границей окна.
    for out_ch, in_ch in zip(a, a[len(b):]):
        # Двигаем окно вправо.
        w_sum -= CH_HASH[ord(out_ch)]
        w_sum += CH_HASH[ord(in_ch)]
        offset += 1
        # И снова проверяем.
        if check(offset, w_sum): 
            return True
            
    return False

check_inv('abcrotm','tro')

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [2]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left is None and self.right is None:
            # Потомков нет, те мы лист, возвращаем значение.
            yield self.value
        else:
            # Есть потомки, мы внутренний узел. 
            if self.left is not None:
                # Возвращаем все левые листовые значения. 
                for value in self.left:
                    yield value                
            if self.right is not None:
                # Возвращаем все правые листовые значения. 
                for value in self.right:
                    yield value
                    
    def __str__(self):
        # __str__ возвращает удобочитаемое представление дерева.
        # T(Значение, левое поддерево/листовое значение, правое ...)
        # 'T(0,T(1,3,4),2)'
        value = str(self.value)
        left = str(self.left) if self.left is not None else ""
        right = str(self.right) if self.right is not None else ""
        if left or right:
            return "T(%s,%s,%s)" % (value, left, right)
        return value
    
    def __repr__(self):
        # __repr__ возвращает максимально однозначное представление дерева.
        # Tree(Значение, левое поддерево, правое поддерево)
        # Tree(0, Tree(1, Tree(3, None, None), Tree(4, None, None)), Tree(2, None, None))
        return "Tree(%r, %s, %s)" % (self.value, repr(self.left), repr(self.right))
            
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [3]:
import re

# Функции вида eval_xxx(expr), вычисляют выражение типа xxx представленное
# префиксом строки expr и возвращают пару (значение, остаток строки expr).
# Предполагается что из выражения удалены пробелы (см. calc).

def eval_number(expr):
    """
        Вычисляет целочисленную константу без знака.
    """
    m = re.match("^(\d+)(.*)$", expr)
    # выражение всегда удволетворяет шаблону, тк eval_factor (единственное место 
    # откуда вызывается eval_number) проверяет что первый символ expr - цифра.
    # Но все что может сломаться, однажды ломается, поэтому проверяем.
    if not m:
        raise SyntaxError("Bad number: '%s'" % expr)
    value, tail = m.groups()    
    return int(value), tail

def eval_factor(expr):
    """ 
        Вычисляет "множитель": число или выражение в скобках с 
        факультативными унарными +-. 
    """
    # Знак выражения.
    sign = 1
    while expr and expr[0] in "+-":
        if expr[0] == "-":
            sign *= -1
        expr = expr[1:]
    if not expr:
        raise SyntaxError("Unexpected end of expression.")
    if expr[0].isdigit():
        value, tail = eval_number(expr)
    elif expr[0] == "(":
        value, tail = eval_expression(expr[1:])
        if not tail:
            raise SyntaxError("Unexpected end of expression.")
        if tail[0] != ")":
            raise SyntaxError("Closed bracket expected befor '%s'." % tail)
        tail = tail[1:]
    return value * sign, tail        
        
    
def eval_term(expr):
    """ Вычисляет "произведение" множителей. """
    value, tail = eval_factor(expr)
    while tail and tail[0] in "*/":
        op = tail[0]
        rvalue, tail = eval_factor(tail[1:])
        if op == "*":
            value *= rvalue
        else:
            value /= rvalue
    return value, tail        
    
def eval_expression(expr):
    """ 
        Вычисляет выражение общего вида: "сумму" произведений. 
    """
    value, tail = eval_term(expr)
    while tail and tail[0] in "+-":
        op = tail[0]
        rvalue, tail = eval_term(tail[1:])
        if op == "+":
            value += rvalue
        else:
            value -= rvalue 
    return value, tail    
    
def calc(expr):
    # Проверяем что в выражении нет целых чисел разделенных 
    # пробельными символами.
    # Если есть, то это синтаксическая ошибка.
    m = re.search(r"\d+\s+\d+", expr, re.MULTILINE)
    if  m:
         raise SyntaxError("Bad number: '%s'" % expr[m.start():m.end()])
    # Теперь можно безопасно удалить из выражения все пробельные символы.     
    expr = re.sub("\s+", "", expr, flags=re.MULTILINE)
    # Вычисляем выражение.
    value, tail = eval_expression(expr)
    # Выражение должно закончится.
    if tail:
        raise SyntaxError("Garbage symbols after the end of expression: '%s'" % tail)
    return value
        
calc('2 * (15 - 3 * 4) - 2') == 4

True